## Utilisation de la libraire `stats` de Scipy pour les extrêmes 

*Attention* : `genextreme.pdf(x, c, loc, scale)` = $f_{GEV}(x|\mu, \sigma, -\xi)$  

In [1]:
import numpy as np
from sympy import *
from sympy.stats import *
import matplotlib.pyplot as plt

from random_variable import GEV_Variable

In [2]:
X = GEV_Variable()
pprint(X.get_cdf())

⎧          μ - x                     
⎪          ─────                     
⎪           sig                      
⎪        -ℯ                          
⎪       ℯ                 for gam = 0
⎪                                    
⎨                    -1              
⎪                    ───             
⎪                    gam             
⎪  ⎛gam⋅(-μ + x)    ⎞                
⎪ -⎜──────────── + 1⎟                
⎪  ⎝    sig         ⎠                
⎩ℯ                         otherwise 


In [3]:
print(X.def_set())
X.eval_params(Mu=0, Sig=1, Gam=1)
print(X.def_set())
print(X.eval_cdf(5))

Piecewise((Interval(-oo, mu - sig/gam), gam < 0), (Interval(-oo, oo), Eq(gam, 0)), (Interval(mu - sig/gam, oo), True))
Interval(-1.00000000000000, oo)
0.846481724890614


In [4]:
X.reparameterize()
X.eval_params(Gam=0)
print(X.eval_cdf(5))

exp(-exp((mu - 5)/sig))


In [5]:
R = X.sympy_rv()
print(P(R>0))
X.eval_params(Mu=0, Sig=1)
R = X.sympy_rv()
print(P(R>0))

exp(mu/sig)*Integral(exp(-_z/sig)*exp(-exp(-_z/sig)*exp(mu/sig)), (_z, 0, oo))/sig
-1.0*exp(-1) + 1.0
